In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_hardware.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_hardware.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_hardware.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_hardware.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'00101111100011101000': 200000}
noise est dist:  {'00101111100011101000': 0.33151, '00101111101011101000': 0.05753, '00101111100011001000': 0.023405, '00111111100111101000': 0.000975, '00101111100011111000': 0.064355, '00111111100010101000': 0.00084, '00111111100011101000': 0.02065, '00101111110011101000': 0.01974, '00101111000011101000': 0.018755, '00101111101011001000': 0.00404, '00101111100111101000': 0.015275, '00101100100011101000': 0.00101, '00101111111011101000': 0.005635, '00101111100001101010': 0.00061, '10101111100011100000': 0.000435, '00101111100011101010': 0.009525, '00101011100111101000': 0.000295, '00111111100011001000': 0.00156, '00101111101010101000': 0.002505, '00101101101011101001': 0.00011, '00101111100000111000': 0.000285, '00111101100011101000': 0.001895, '01101111100011101000': 0.007785, '10101111100011101100': 0.000465, '00101111100001101000': 0.020505, '00101111100001001000': 0.001635, '00001110100011101000': 0.00022, '10101111100011101000': 0.01204, '

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

# if 2**10<2**num_qubits:
num_keep=2**15#num_qubits#2**10
if num_keep>2**num_qubits:
    num_keep=2**num_qubits
noisy_counts_orig=deepcopy(noisy_counts)
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}



num keep  32768
max exist  1033219
min exist  0
size before  17156
32768
32768


In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=20, 
               legend=["ideal", "raw", "corrected"], title="Dicke n=20, k=1 Quantum Hardware", bar_labels=False)
fig.savefig("plot_dicke20_1_counts_hardware_ibm_marrakesh.jpg", bbox_inches="tight")


In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=20, 
                  legend=["ideal", "raw", "corrected"], title="Dicke n=20, k=1 Quantum Hardware", bar_labels=False) #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.savefig("plot_dicke20_1_dist_hardware_ibm_marrakesh.jpg", bbox_inches="tight")

In [7]:
from qiskit.quantum_info import hellinger_fidelity
print(hellinger_fidelity(ideal_counts, quasi_corrected_counts))
print(hellinger_fidelity(ideal_counts, noisy_counts_orig))
print(ideal_counts)
print(quasi_corrected_counts)
print(noisy_counts_orig)

0.8026178283809842
0.282890350465598
{'10000000000000000000': 10000, '01000000000000000000': 10000, '00100000000000000000': 10000, '00010000000000000000': 10000, '00001000000000000000': 10000, '00000100000000000000': 10000, '00000010000000000000': 10000, '00000001000000000000': 10000, '00000000100000000000': 10000, '00000000010000000000': 10000, '00000000001000000000': 10000, '00000000000100000000': 10000, '00000000000010000000': 10000, '00000000000001000000': 10000, '00000000000000100000': 10000, '00000000000000010000': 10000, '00000000000000001000': 10000, '00000000000000000100': 10000, '00000000000000000010': 10000, '00000000000000000001': 10000}
{'00000011010111100110': 0.894671899829518, '10000000000000110000': 3.84362895876671, '10000000100010110000': 8.71105196453908, '00001000110001000110': 15.4881453598924, '01001000000000000000': 16.0495251099057, '10000010100000000000': 18.3911203912864, '10000010000000000000': 21.6134194739312, '00000000101011111010': 26.2297592374412, '000